# WSF/JSE/JS/VBA/VBE File Execution
Detects suspicious file execution by wscript and cscript

## Rule Content
```
- title: WSF/JSE/JS/VBA/VBE File Execution
  id: 1e33157c-53b1-41ad-bbcc-780b80b58288
  status: experimental
  description: Detects suspicious file execution by wscript and cscript
  author: Michael Haag
  tags:
  - attack.execution
  - attack.t1064
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image:
      - '*\wscript.exe'
      - '*\cscript.exe'
      CommandLine:
      - '*.jse'
      - '*.vbe'
      - '*.js'
      - '*.vba'
    condition: selection
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Will need to be tuned. I recommend adding the user profile path in CommandLine
    if it is getting too noisy.
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:(*\\wscript.exe OR *\\cscript.exe) AND process_command_line.keyword:(*.jse OR *.vbe OR *.js OR *.vba))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()